### Predição de propriedades moleculares com aprendizado de máquina

O projeto busca aplicar técnicas de aprendizado de máquina para otimização de processos laboratoriais. Para isso, contamos com o carregamento de dados experimentais do dataset [QM9](http://quantum-machine.org/datasets/), o qual disponibiliza a informação da energia **homo - lumo**, o **momento de dipolo** e o número de átomos de mais de 130000 moléculas orgânicas, indexadas pelos respectivos SMILES, as quais são formadas pelos átomos de carbono, oxigênio, fluor e nitrogênio.

In [1]:
#library import

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns 
from rdkit import Chem

# machine learning library
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#local module import
import sys
import os
sys.path.append(os.path.abspath("/home/edmurcn/Documentos/MeusProjetos/Predicao_Molecular")) 
from src.modeling_utils import DataIngestion, DataIngestion_pipe
from src.model_train2 import evaluate_models_cv_regression

In [2]:
# data input

df_mol = pd.read_csv("/home/edmurcn/Documentos/MeusProjetos/Predicao_Molecular/data/origin_dataset.csv", header=0)

print("Número de moléculas:", df_mol.shape[0])
print(50*"-")
print("Colunas:")
df_mol.head(0)


Número de moléculas: 133885
--------------------------------------------------
Colunas:


,smiles,n_atoms,homo_lumo_nm,dipole_moment_debye


In [3]:
# Checking if there is NaN values

df_mol.isna().sum()

smiles                 0
n_atoms                0
homo_lumo_nm           0
dipole_moment_debye    0
dtype: int64

#### **Tratamento e ingestão de dados**

Nesta etapa aplicamos uma linha de processamento contida na classe **DataIngestion()** para gerar um novo conjunto de dados, os processos são apresentados abaixo:

- **drop_duplicated()** : Método utilizado para retirar moléculas duplicadas do conjunto de dados original. De acordo com as referências do projeto, existe a possibilidade de a mesma molécula ser escrita com SMILES diferentes, como 'C=CCC' e 'CCC=C', apesar de representarem a mesma estrutura, são escritos de maneira diferente devido a simetria.

- **get_features()** : Método utilizado para gerar características das moléculas e adicionar ao conjunto original. As novas características serão geradas através do resultado de todos os descritores presentes na biblioteca RDKit (biblioteca voltada a química computacional).

- **get_best_features()**: Método que seleciona as características com melhor perfil para serem usadas em dados de predição. Buscamos selecionar as features de acordo com o valor de correlação e da variância, dado que colunas com grandes correlações ou pequena variância foram excluídas.

Por fim, os métodos podem ser executados separadamente ou, ao utilizar a classe **DataIngestion_pipe()**, temos a execução em sequência de todos os métodos

In [ ]:
# The code is commented because the data was created and saved in the path, so just run it to create new data

# data_ingestion = DataIngestion_pipe()
# data_ingestion.fit_transform(X=df_mol)

In [13]:
df_mol_new = pd.read_csv("/home/edmurcn/Documentos/MeusProjetos/Predicao_Molecular/data/data_best_features.csv", header=0)
print("Número de moléculas:", df_mol_new.shape[0])
print(50*"-")
print("Novas colunas:", df_mol_new.shape[1])
df_mol_new.head(0)

Número de moléculas: 133798
--------------------------------------------------
Novas colunas: 141


,smiles,n_atoms,homo_lumo_nm,dipole_moment_debye,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,MaxPartialCharge,...,fr_para_hydroxylation,fr_phenol,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_term_acetylene,fr_tetrazole,fr_urea


#### **Dicionário dos dados**

In [7]:
# Save and dropping homo_lumo columns - because is a highly correlated with dipole columns and its a possible target in the others preditions
# Save and dropping smiles and dipole_moment_debye columns

homo_lumo = df_mol_new["homo_lumo_nm"]

smile_and_target = df_mol_new[["smiles", "dipole_moment_debye"]]

df_mol_new = df_mol_new.drop(["homo_lumo_nm", "smiles", "dipole_moment_debye"], axis=1)


#### **Redução de dimensionalidade**

Nesta etapa testamos alguns métodos para reduzir ainda mais a dimensão dos dados. Serão aplicados métodos como **PCA()**, **LDA()**... em uma amostra aleatória, contendo 10000 instẫncias dos dados originais.

- **PCA** (*Principal Components Analysis*): O componente principal é um vetor unitário compostos por contribuições de cada feature do dataset, com esses vetores é possível formar hiperplanos e assim realizar a projeção dos dados para obter o máximo de variância explicada (informação) no mínimo de dimensões possíveis.

In [8]:
# Create the train and test datasets and standardize scale

scaler = StandardScaler()

x_train, x_test, y_train, y_test = train_test_split(df_mol_new, smile_and_target["dipole_moment_debye"], test_size=0.2, random_state=0)

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train.shape
y_train.shape

(107038,)

In [9]:
# Apply PCA methods to explain more than 95% of data variance

pca = PCA(n_components=0.95)

x_train_reduced = pca.fit_transform(x_train)
x_test_reduced = pca.transform(x_test)

print("Número de componentes principais:", x_train_reduced.shape[1])
print(50*"-")
print("Contribuição em taxa de variância explicada de cada componente:")
print(50*"-")
print(pca.explained_variance_ratio_ )

Número de componentes principais: 70
--------------------------------------------------
Contribuição em taxa de variância explicada de cada componente:
--------------------------------------------------
[0.15597672 0.06137889 0.05156915 0.04584766 0.03750378 0.03442346
 0.03073575 0.02629401 0.02038951 0.01985551 0.01950092 0.01704463
 0.01662826 0.01540467 0.01468783 0.01371115 0.01289675 0.01281101
 0.01188442 0.0113059  0.01092915 0.01062289 0.01036587 0.00989117
 0.00951487 0.00924796 0.00867506 0.00848655 0.00833669 0.00812761
 0.0080102  0.00786781 0.00764632 0.00750701 0.00740467 0.00734956
 0.00726372 0.00726053 0.00723058 0.00720648 0.00715004 0.00702522
 0.00688086 0.00681215 0.00663761 0.00657208 0.00628369 0.00623896
 0.00595841 0.00585417 0.00576835 0.00568229 0.00545408 0.00531152
 0.00502969 0.00498392 0.00494856 0.00486765 0.00469256 0.00447829
 0.00409456 0.00390719 0.00380737 0.00378078 0.00371903 0.00342495
 0.00325135 0.0031904  0.0031396  0.00294138]


Notamos que o método de PCA desempenha muito bem a redução de dimensionalidade, porém, dentre os componentes principais obtidos, cada um contribui com uma baixa taxa de variância explicada, evidenciando que não uma coluna que diferencie com eficácia as amostras. Logo, deve-se avaliar no processo de treinamento se o tempo ganho com a redução de dimensionalidade compensa a informação perdida ao realizar o método.

## Realizar a exploração dos dados